# Pandas 2.2.2用

## 0) 前提

* 環境: **Python 3.10.15 / pandas 2.2.2**
* **指定シグネチャ厳守**（本問題は単一 DF 入力のため 1 引数）
* I/O 禁止、不要な `print` や `sort_values` 禁止

## 1) 問題

* `面積が 3,000,000 以上 または 人口が 25,000,000 以上の国（big）を抽出し、name, population, area を返す`
* 入力 DF: `World(name, continent, area, population, gdp)`
* 出力: `name, population, area`（順不同・列順固定）

## 2) 実装（指定シグネチャ厳守）

> 列最小化 → 条件抽出のみで十分（グループ処理不要）。

```python
import pandas as pd

def big_countries(World: pd.DataFrame) -> pd.DataFrame:
    """
    Args:
        World (pd.DataFrame): 列 name, continent, area, population, gdp を含む
    Returns:
        pd.DataFrame: 列名と順序は ['name', 'population', 'area']
    """
    mask = (World['area'] >= 3_000_000) | (World['population'] >= 25_000_000)
    out = World.loc[mask, ['name', 'population', 'area']]
    return out

Analyze Complexity
Runtime 272 ms
Beats 52.01%
Memory 68.78 MB
Beats 67.98%

```

## 3) アルゴリズム説明

* 使用 API

  * ブールインデクシング：`(df['col'] >= x) | (df['col'] >= y)` による条件抽出
  * 列最小化：`df.loc[mask, ['name','population','area']]`
* **NULL / 重複 / 型**

  * `NaN >= 閾値` は `False` になるため、`area`/`population` が欠損の行は自動的に除外される
  * 入力は主キー `name` 前提のため重複は想定せず（存在しても処理は安定）
  * 数値列が文字列として来る場合は事前に `to_numeric` で正規化するのが安全（本実装では前提どおり数値型として扱う）

## 4) 計算量（概算）

* 条件評価とフィルタ：**O(N)**（列走査）
* 追加メモリ：ブールマスクと出力列分で **O(N)**

## 5) 図解（Mermaid 超保守版）

```mermaid
flowchart TD
  A[入力 データフレーム World] --> B[列最小化 指定列のみ]
  B --> C[条件抽出 area>=3000000 <br>または population>=25000000]
  C --> D[出力 name population area]
```

この問題は処理自体が軽いので“劇的”な伸びは出にくいですが、**数％〜十数％**の改善が狙える小技はあります。下のどれか一つでも効けば御の字です。

## マイクロ最適化のポイント

1. **NumExpr を使って一時配列を減らす**（OR 条件は効きやすい）

```python
import pandas as pd

def big_countries(World: pd.DataFrame) -> pd.DataFrame:
    # dtype を触らない純粋最適化：NumExpr で中間配列を減らす
    mask = pd.eval(
        "(area >= 3000000) | (population >= 25000000)",
        engine="numexpr",  # numexpr が入っていれば自動使用
        target=World
    )
    return World.loc[mask, ['name', 'population', 'area']]
```

* `pd.eval(..., engine='numexpr')` はブール式の中間オブジェクトを最小化して CPU/L1/L2 ミスを減らします。

2. **dtype を軽量化（I/O とメモリ圧を軽減）**

```python
def big_countries(World: pd.DataFrame) -> pd.DataFrame:
    # 可能なら 32bit 化（上限が収まる前提）
    area = World['area'].astype('int32', copy=False)
    pop  = World['population'].astype('int32', copy=False)

    mask = (area >= 3_000_000) | (pop >= 25_000_000)
    return World.loc[mask, ['name', 'population', 'area']]

Analyze Complexity
Runtime 260 ms
Beats 75.21%
Memory 68.72 MB
Beats 67.98%

```

* 32bit で十分な列は `astype('int32', copy=False)` にするだけで **メモリ↓** → **キャッシュ効率↑**。
* ただし欠損があるなら `Int32`（nullable）を検討。

3. **Copy-on-Write を有効化**（Pandas 2.2 系の地味に効く設定）

```python
pd.options.mode.copy_on_write = True
```

* 余計なコピーを減らせるケースがあり、**メモリ使用量がじわっと改善**。

4. **NumPy 直アクセスでマスク生成を最短経路に**

```python
import numpy as np

def big_countries(World: pd.DataFrame) -> pd.DataFrame:
    area = World['area'].to_numpy()        # 通常はゼロコピー（dtype整合時）
    pop  = World['population'].to_numpy()
    mask = (area >= 3_000_000) | (pop >= 25_000_000)
    # iloc で行だけ切り出してから列を最小化
    out = World.iloc[np.flatnonzero(mask)][['name', 'population', 'area']]
    return out

Analyze Complexity
Runtime 243 ms
Beats 95.53%
Memory 69.18 MB
Beats 21.49%

```

* `to_numpy()` は dtype が適合していれば**ビュー**になり追加メモリを使いません。

---

### どれを採用すべき？

* まずは **(1) NumExpr** と **(3) CoW** を試してみる → 体感で速くなることが多いです。
* 次に **(2) dtype 軽量化**（`int32` 化）が可能なら適用。
* さらに突き詰めるなら **(4) NumPy 直アクセス** 版。

> いずれも API 互換は保ち、**I/O・print・sort なし**、列順も仕様どおりです。
> 実データ規模・dtype・欠損の有無で効き方は変わるので、ベンチを 3 回ずつ計測して中央値で比較するのがオススメです。

エラーの原因は、提案した `pd.eval(..., engine="numexpr")` を使う最適化に対して **numexpr が未インストール**（または非対応バージョン）だからです。対処は次のいずれかです。

---

## A) numexpr を入れて最適化を活かす

**pip（仮想環境）**

```bash
python -m pip install -U numexpr
```

**conda**

```bash
conda install -c conda-forge numexpr
```

> 参考: pandas 2.2.2 / Python 3.10 なら `numexpr` は対応しています。
> 追加で高速化するなら環境変数 `NUMEXPR_MAX_THREADS` を CPU 論理コア数に設定しておくと良いです。

---

## B) ランタイム制約でインストール不可なら、フォールバック実装に変更

`numexpr` 依存をやめて **純 pandas / NumPy** のベクトル演算に戻せば、環境に依存せず動きます（元の速度とほぼ同等〜少し上）。

```python
import pandas as pd
import numpy as np

def big_countries(World: pd.DataFrame) -> pd.DataFrame:
    """
    Returns:
        pd.DataFrame: ['name', 'population', 'area']
    """
    # dtype が数値なら to_numpy() は基本ゼロコピーで高速
    area = World['area'].to_numpy()
    pop  = World['population'].to_numpy()
    mask = (area >= 3_000_000) | (pop >= 25_000_000)
    return World.iloc[np.flatnonzero(mask)][['name', 'population', 'area']]
```

---

## C) 自動判定で numexpr があれば使い、なければフォールバック

```python
import pandas as pd

def big_countries(World: pd.DataFrame) -> pd.DataFrame:
    try:
        import numexpr  # noqa: F401
        mask = pd.eval(
            "(area >= 3000000) | (population >= 25000000)",
            engine="numexpr",
            target=World
        )
    except Exception:
        # numexpr なし or 非対応 → 通常のベクトル演算にフォールバック
        mask = (World['area'] >= 3_000_000) | (World['population'] >= 25_000_000)

    return World.loc[mask, ['name', 'population', 'area']]

Analyze Complexity
Runtime 261 ms
Beats 73.67%
Memory 68.91 MB
Beats 46.36%

```

---

### どれを使うべき？

* **インストールできる**なら **A**（最速化の余地）。
* インストールが難しい/オンライン禁止なら **B**（安定・高速）。
* 同一コードをどの環境でも使いたいなら **C**（自動判定）。

> なお、`pd.eval(..., engine="python")` に切り替える方法もありますが、速度面の利点が薄く、ベクトル演算（B）のほうがシンプルで安定です。

